In [1]:
#Import the packages that I will use 
import os
import json
import numpy as np
import pandas as pd
import pprint 
import mysql.connector
import pymysql
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import requests
import datetime
import schedule
import time

In [2]:
# Database Creation: 
# I first connect to a MySQL database, drops a database named 'project2database' (if it exists), and then creates it again.
connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Uva!1819",
    port="3306"
)

cursor = connection.cursor()

database_drop = 'project2database'
try:
    cursor.execute(f"DROP DATABASE {database_drop}")
    print(f"Database '{database_drop}' dropped successfully!")
except mysql.connector.Error as err:
    print(f"Error: {err}")

try:
    cursor.execute("CREATE DATABASE project2database")
    print("Database 'project2database' created successfully!")
except mysql.connector.Error as err:
    print(f"Error: {err}")
finally:
    cursor.close()
    connection.close()

Database 'project2database' dropped successfully!
Database 'project2database' created successfully!


In [3]:
#Then I create the SQL database that will house all the data from the api call
def create_table():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Uva!1819",
        database="project2database",
        port="3306"
    )
    cursor = connection.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS api_data (
            id INT AUTO_INCREMENT PRIMARY KEY,
            factor INT,
            pi DOUBLE,
            time DATETIME
        )
    """)
    print("Table 'api_data' created successfully!")
    cursor.close()
    connection.close()

create_table()


Table 'api_data' created successfully!


In [ ]:
#fetch_data_from_api is a function that makes an HTTP GET request to the API URL. If the request is successful (status_code == 200), it returns the JSON response; otherwise, it prints an error message. 
def fetch_data_from_api():
    api_url = "https://4feaquhyai.execute-api.us-east-1.amazonaws.com/api/pi"
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: Status code {response.status_code}")
        return None
#I then insert the data from the call into the sql table I made in my dataset
def insert_data_to_database(data):
    query = "INSERT INTO api_data (factor, pi, time) VALUES (%s, %s, %s)"
    values = (data['factor'], data['pi'], data['time'])


    connection = pymysql.connect(host='localhost', user='root', password='Uva!1819', database='project2database')
    cursor = connection.cursor()
    try:
        cursor.execute(query, values)
        connection.commit()
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")
    finally:
        cursor.close()
        connection.close()


#next_run_time calculates the start of the next minute based on the current system time. This is used to schedule tasks at the start of each minute.
def next_run_time():
 
    now = datetime.datetime.now()
    return now.replace(second=0, microsecond=0) + datetime.timedelta(minutes=1)
#scheduled_job fetches data from the API using fetch_data_from_api and, if successful, inserts the data into the database using insert_data_to_database
def scheduled_job():
    data = fetch_data_from_api()
    if data:
        insert_data_to_database(data)
#run_for_an_hour runs a loop that continues for one hour. Inside the loop, it checks if the current time has reached or passed the next_run time. If so, it executes scheduled_job and recalculates the next_run time.


def run_for_an_hour():
    now = datetime.datetime.now()
    start_time = now.replace(minute=0, second=0, microsecond=0)
    if now.minute > 0 or now.second > 0 or now.microsecond > 0:
        start_time = start_time + datetime.timedelta(hours=1)
    end_time = start_time + datetime.timedelta(hours=1)
    while datetime.datetime.now() < start_time:
        pass  
    next_run = start_time
    while datetime.datetime.now() < end_time:
        current_time = datetime.datetime.now()
        if current_time >= next_run:
            scheduled_job()
            next_run = current_time.replace(second=0, microsecond=0) + datetime.timedelta(minutes=1)

    print("Execution completed. Exiting after 60 minutes.")

#This is the main entry point of the script. When the script is executed, it calls run_for_an_hour to start the process.
if __name__ == "__main__":
    run_for_an_hour()



In [ ]:
#The data trend shows that cubing each minute of a timestamp (starting from 1^3 at 00 minutes to 59^3 at 59 minutes) is used to approximate π (Pi). The approximations become increasingly accurate as the cube values grow larger with each passing minute. The most precise approximation of π occurs at the 59-minute mark, highlighting a clear correlation between the magnitude of the cube root and the accuracy of the π approximation. 